In [52]:
import pandas as pd
train_data='C:/Users/82106/Desktop/데이터분석 프로젝트/대구 교통사고 피해 예측 AI 경진대회/open/open/train.csv'
test_data='C:/Users/82106/Desktop/데이터분석 프로젝트/대구 교통사고 피해 예측 AI 경진대회/open/open/test.csv'
train_df = pd.read_csv(train_data)
test_df = pd.read_csv(test_data)
train_df.head()
test_df.head()

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형
0,ACCIDENT_39609,2022-01-01 01,토요일,맑음,대구광역시 수성구 상동,교차로 - 교차로안,건조,차대사람
1,ACCIDENT_39610,2022-01-01 01,토요일,맑음,대구광역시 수성구 지산동,단일로 - 기타,건조,차대사람
2,ACCIDENT_39611,2022-01-01 04,토요일,맑음,대구광역시 수성구 수성동2가,교차로 - 교차로안,건조,차대차
3,ACCIDENT_39612,2022-01-01 04,토요일,맑음,대구광역시 수성구 신매동,단일로 - 기타,건조,차대차
4,ACCIDENT_39613,2022-01-01 06,토요일,맑음,대구광역시 달서구 감삼동,교차로 - 교차로안,건조,차대차


In [53]:
# 결측치 확인
train_missing_values = train_df.isnull().sum()
train_missing_values[train_missing_values > 0]


피해운전자 차종      991
피해운전자 성별      991
피해운전자 연령      991
피해운전자 상해정도    991
dtype: int64

In [54]:
# 각 변수의 가장 빈번한 값 찾기
most_frequent_values = train_df[['피해운전자 차종', '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도']].mode().iloc[0]
most_frequent_values

# 결측치를 가장 빈번한 값으로 대체
train_df_filled = train_df.fillna(most_frequent_values)

# 대체 후 결측치 확인
missing_values_after_filling = train_df_filled.isnull().sum()
missing_values_after_filling, most_frequent_values


(ID             0
 사고일시           0
 요일             0
 기상상태           0
 시군구            0
 도로형태           0
 노면상태           0
 사고유형           0
 사고유형 - 세부분류    0
 법규위반           0
 가해운전자 차종       0
 가해운전자 성별       0
 가해운전자 연령       0
 가해운전자 상해정도     0
 피해운전자 차종       0
 피해운전자 성별       0
 피해운전자 연령       0
 피해운전자 상해정도     0
 사망자수           0
 중상자수           0
 경상자수           0
 부상자수           0
 ECLO           0
 dtype: int64,
 피해운전자 차종       승용
 피해운전자 성별        남
 피해운전자 연령      59세
 피해운전자 상해정도     경상
 Name: 0, dtype: object)

In [55]:
from sklearn.preprocessing import OneHotEncoder

# 원-핫 인코딩을 적용할 범주형 변수 선택
categorical_features = ['요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형','법규위반']

# 원-핫 인코더 초기화
onehot_encoder = OneHotEncoder(sparse=False)

# 원-핫 인코딩 적용
encoded_features = onehot_encoder.fit_transform(train_df_filled[categorical_features])

# 인코딩된 특성의 열 이름 가져오기
encoded_feature_names = onehot_encoder.get_feature_names_out(categorical_features)

# 원-핫 인코딩된 특성을 DataFrame으로 변환
encoded_df = pd.DataFrame(encoded_features, columns=encoded_feature_names)

# 원본 데이터프레임에서 범주형 변수 제거 후 인코딩된 특성 추가
train_df_ready = train_df_filled.drop(categorical_features, axis=1).join(encoded_df)

train_df_ready.head()


,ID,사고일시,사고유형 - 세부분류,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,...,법규위반_교차로운행방법위반,법규위반_기타,법규위반_보행자보호의무위반,법규위반_불법유턴,법규위반_신호위반,법규위반_안전거리미확보,법규위반_안전운전불이행,법규위반_중앙선침범,법규위반_직진우회전진행방해,법규위반_차로위반
0,ACCIDENT_00000,2019-01-01 00,길가장자리구역통행중,승용,여,51세,상해없음,보행자,여,70세,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,ACCIDENT_00001,2019-01-01 00,보도통행중,승용,남,39세,상해없음,보행자,남,61세,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ACCIDENT_00002,2019-01-01 01,차도통행중,승용,남,70세,상해없음,보행자,남,38세,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,ACCIDENT_00003,2019-01-01 02,추돌,승용,남,49세,상해없음,승용,남,36세,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,ACCIDENT_00004,2019-01-01 04,추돌,승용,남,30세,상해없음,승용,남,52세,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [41]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# 타겟 변수와 특성 변수 분리
X = train_df_ready.drop(['ID', '사고일시', '사고유형 - 세부분류', '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종', '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', 'ECLO'], axis=1)
y = train_df_ready['ECLO']

# 훈련 세트와 테스트 세트 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 선형 회귀 모델 생성 및 훈련
model = LinearRegression()
model.fit(X_train, y_train)

# 테스트 세트에서 성능 평가
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

mse


1.7641862925564254e-28

In [61]:
# Removing '법규위반' from the list of categorical features for the test data
categorical_features_test = ['요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형']


In [62]:
# Creating a new OneHotEncoder instance for the test data
onehot_encoder_test = OneHotEncoder(sparse=False)

# Fitting and transforming the test data with the new encoder
encoded_test_features_new = onehot_encoder_test.fit_transform(test_df_filled[categorical_features_test])
encoded_feature_names_new = onehot_encoder_test.get_feature_names_out(categorical_features_test)
encoded_test_df_new = pd.DataFrame(encoded_test_features_new, columns=encoded_feature_names_new)

# Preparing the test data by adding encoded features and removing original categorical features
test_df_ready_new = test_df_filled.drop(categorical_features_test, axis=1).join(encoded_test_df_new)

# Aligning the columns of the test data with those of the training data
# This step is crucial to ensure that the model receives the same number of features during prediction
test_df_ready_new = test_df_ready_new.reindex(columns=X_train.columns, fill_value=0)

# Predicting using the trained model
y_pred_test_new = model.predict(test_df_ready_new)

# Displaying the first few predictions
y_pred_test_new[:10]


array([0.01918548, 0.01918548, 0.01918548, 0.01918548, 0.01918548,
       0.01918548, 0.01918548, 0.01918548, 0.01918548, 0.01918548])

### XGBoost

###### test 데이터 전처리 진행

In [85]:
import pandas as pd
train_data='C:/Users/82106/Desktop/데이터분석 프로젝트/대구 교통사고 피해 예측 AI 경진대회/open/open/train.csv'
test_data='C:/Users/82106/Desktop/데이터분석 프로젝트/대구 교통사고 피해 예측 AI 경진대회/open/open/test.csv'
train_df = pd.read_csv(train_data)
test_df = pd.read_csv(test_data)

In [86]:
# 결측치 처리: 가장 빈번한 값으로 결측치를 대체합니다.
# 먼저 가장 빈번한 값을 찾습니다.
most_frequent_values = train_df.mode().iloc[0]

# 이제 훈련 데이터와 테스트 데이터에 결측치를 대체합니다.
train_df_filled = train_df.fillna(most_frequent_values)
test_df_filled = test_df.fillna(most_frequent_values)


In [89]:
# 원-핫 인코딩 적용
# 원-핫 인코더를 초기화하고 훈련 데이터에 적용합니다.
onehot_encoder = OneHotEncoder(sparse=False)
encoded_train_features = onehot_encoder.fit_transform(train_df_filled[categorical_features])


In [90]:
# 인코딩된 특성을 훈련 데이터에 추가하고 원래 범주형 변수 제거
encoded_train_df = pd.DataFrame(encoded_train_features, columns=onehot_encoder.get_feature_names_out(categorical_features))
train_df_ready = train_df_filled.drop(categorical_features, axis=1).join(encoded_train_df)

In [93]:
# '법규위반'을 제외한 범주형 변수 목록
categorical_features_without_violation = ['요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형']

# 원-핫 인코더 재설정
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoder.fit(train_df_filled[categorical_features_without_violation])

# 훈련 데이터에 대해 원-핫 인코딩 적용
encoded_train_features = onehot_encoder.transform(train_df_filled[categorical_features_without_violation])
encoded_train_df = pd.DataFrame(encoded_train_features, columns=onehot_encoder.get_feature_names_out(categorical_features_without_violation))
train_df_ready = train_df_filled.drop(categorical_features_without_violation, axis=1).join(encoded_train_df)

# 테스트 데이터에 대해 원-핫 인코딩 적용
encoded_test_features = onehot_encoder.transform(test_df_filled[categorical_features_without_violation])
encoded_test_df = pd.DataFrame(encoded_test_features, columns=onehot_encoder.get_feature_names_out(categorical_features_without_violation))
test_df_ready = test_df_filled.drop(categorical_features_without_violation, axis=1).join(encoded_test_df)

##### train 데이터 다시 전처리

In [98]:
# 결측치 확인
train_missing_values = train_df.isnull().sum()
train_missing_values[train_missing_values > 0]


피해운전자 차종      991
피해운전자 성별      991
피해운전자 연령      991
피해운전자 상해정도    991
dtype: int64

In [99]:
# 각 변수의 가장 빈번한 값 찾기
most_frequent_values = train_df[['피해운전자 차종', '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도']].mode().iloc[0]
most_frequent_values

# 결측치를 가장 빈번한 값으로 대체
train_df_filled = train_df.fillna(most_frequent_values)

# 대체 후 결측치 확인
missing_values_after_filling = train_df_filled.isnull().sum()
missing_values_after_filling, most_frequent_values

(ID             0
 사고일시           0
 요일             0
 기상상태           0
 시군구            0
 도로형태           0
 노면상태           0
 사고유형           0
 사고유형 - 세부분류    0
 법규위반           0
 가해운전자 차종       0
 가해운전자 성별       0
 가해운전자 연령       0
 가해운전자 상해정도     0
 피해운전자 차종       0
 피해운전자 성별       0
 피해운전자 연령       0
 피해운전자 상해정도     0
 사망자수           0
 중상자수           0
 경상자수           0
 부상자수           0
 ECLO           0
 dtype: int64,
 피해운전자 차종       승용
 피해운전자 성별        남
 피해운전자 연령      59세
 피해운전자 상해정도     경상
 Name: 0, dtype: object)

In [103]:
from sklearn.preprocessing import OneHotEncoder

# 원-핫 인코딩을 적용할 범주형 변수 선택
categorical_features = ['요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형','법규위반']

# 원-핫 인코더 초기화
onehot_encoder = OneHotEncoder(sparse=False)

# 원-핫 인코딩 적용
encoded_features = onehot_encoder.fit_transform(train_df_filled[categorical_features])

# 인코딩된 특성의 열 이름 가져오기
encoded_feature_names = onehot_encoder.get_feature_names_out(categorical_features)

# 원-핫 인코딩된 특성을 DataFrame으로 변환
encoded_df = pd.DataFrame(encoded_features, columns=encoded_feature_names)

# 원본 데이터프레임에서 범주형 변수 제거 후 인코딩된 특성 추가
train_df_ready = train_df_filled.drop(categorical_features, axis=1).join(encoded_df)

train_df_ready.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39609 entries, 0 to 39608
Columns: 259 entries, ID to 법규위반_차로위반
dtypes: float64(243), int64(5), object(11)
memory usage: 78.3+ MB


In [104]:
# XGBoost 회귀 모델 생성
xgb_model = xgb.XGBRegressor(objective='reg:squarederror')

# 훈련 데이터에서 비수치형 컬럼을 제외하고 타겟 변수와 특성을 분리
X_train = train_df_ready.select_dtypes(include=['int64', 'float64'])
y_train = train_df_ready['ECLO']

# 모델 훈련
xgb_model.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [109]:
# 테스트 데이터에서 훈련 데이터의 특성에 해당하는 컬럼만을 선택
# 또한 테스트 데이터에 없는 특성에 대해서는 무시
X_test = test_df_ready.reindex(columns=train_features).fillna(0)

# 예측 수행
y_pred_test = xgb_model.predict(X_test)

##### 모델 평가 하기

In [115]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# 훈련 데이터셋을 훈련 및 검증 데이터셋으로 분리
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42)

# 모델 훈련
xgb_model = xgb.XGBRegressor(objective='reg:squarederror')
xgb_model.fit(X_train, y_train)

# 검증 데이터셋에 대한 예측 수행
y_pred_valid = xgb_model.predict(X_valid)

# RMSE 계산
rmse = np.sqrt(mean_squared_error(y_valid, y_pred_valid))
print("RMSE: ", rmse)


RMSE:  0.06701831691265782


In [111]:
# 제출 파일 생성
sample_submission_df = pd.read_csv('C:/Users/82106/Desktop/데이터분석 프로젝트/대구 교통사고 피해 예측 AI 경진대회/open/open/sample_submission.csv')
sample_submission_df['ECLO'] = y_pred_test

# 예측 결과를 CSV 파일로 저장
submission_file = 'C:/Users/82106/Desktop/데이터분석 프로젝트/대구 교통사고 피해 예측 AI 경진대회/open/open/submission_xgboost.csv'
sample_submission_df.to_csv(submission_file, index=False)
# 제출 파일 경로 반환
submission_file

'C:/Users/82106/Desktop/데이터분석 프로젝트/대구 교통사고 피해 예측 AI 경진대회/open/open/submission_xgboost.csv'

### xgboost 오버피팅 확인하기

In [116]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import xgboost as xgb

# 훈련 데이터셋을 훈련 및 검증 데이터셋으로 분리
X_train_full, X_valid, y_train_full, y_valid = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42)

# XGBoost 회귀 모델 생성
xgb_model = xgb.XGBRegressor(objective='reg:squarederror')

# 모델 훈련
xgb_model.fit(X_train_full, y_train_full)

# 훈련 데이터셋과 검증 데이터셋에 대한 예측
y_pred_train = xgb_model.predict(X_train_full)
y_pred_valid = xgb_model.predict(X_valid)

# RMSE 계산
rmse_train = np.sqrt(mean_squared_error(y_train_full, y_pred_train))
rmse_valid = np.sqrt(mean_squared_error(y_valid, y_pred_valid))

print("RMSE on Training set: ", rmse_train)
print("RMSE on Validation set: ", rmse_valid)



RMSE on Training set:  0.00011669734611522376
RMSE on Validation set:  0.11442886412968958


오버피팅이 나타남

##### 교차검증 수행

In [117]:
from sklearn.model_selection import cross_val_score
import numpy as np
import xgboost as xgb

# XGBoost 회귀 모델 생성
xgb_model = xgb.XGBRegressor(objective='reg:squarederror')

# 교차 검증을 사용하여 RMSE 계산
# 여기서는 5-폴드 교차 검증을 사용합니다.
scores = cross_val_score(xgb_model, X_train, y_train, scoring='neg_mean_squared_error', cv=5)

# RMSE 계산
rmse_scores = np.sqrt(-scores)

# 결과 출력
print("RMSE scores:", rmse_scores)
print("Mean RMSE:", rmse_scores.mean())
print("Standard deviation:", rmse_scores.std())


RMSE scores: [0.10533165 0.13279049 0.02103458 0.14079578 0.11731998]
Mean RMSE: 0.10345449605799736
Standard deviation: 0.04299718644754564


오버피팅이 나타나므로 다른 모델적용

### 베이지안 회귀

In [118]:
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# 훈련 데이터셋을 훈련 및 검증 데이터셋으로 분리
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42)

# 베이지안 회귀 모델 생성
bayesian_model = BayesianRidge()

# 모델 훈련
bayesian_model.fit(X_train, y_train)

# 검증 데이터셋에 대한 예측
y_pred_valid = bayesian_model.predict(X_valid)

# RMSE 계산
rmse = np.sqrt(mean_squared_error(y_valid, y_pred_valid))
print("RMSE on Validation set: ", rmse)


RMSE on Validation set:  7.441738090637709e-13


In [119]:
# 훈련 데이터에서 비수치형 컬럼을 제외하고 타겟 변수와 특성을 분리
X_train = train_df_ready.select_dtypes(include=['int64', 'float64'])
y_train = train_df_ready['ECLO']

# 모델 훈련
bayesian_model.fit(X_train, y_train)

BayesianRidge()

In [121]:
# 테스트 데이터에서 훈련 데이터의 특성에 해당하는 컬럼만을 선택
# 또한 테스트 데이터에 없는 특성에 대해서는 무시
X_test = test_df_ready.reindex(columns=train_features).fillna(0)

# 예측 수행
y_pred_test = bayesian_model.predict(X_test)

In [122]:
y_pred_test

array([0.0004621, 0.0004621, 0.0004621, ..., 0.0004621, 0.0004621,
       0.0004621])

In [123]:
# 제출 파일 생성
sample_submission_df = pd.read_csv('C:/Users/82106/Desktop/데이터분석 프로젝트/대구 교통사고 피해 예측 AI 경진대회/open/open/sample_submission.csv')
sample_submission_df['ECLO'] = y_pred_test

# 예측 결과를 CSV 파일로 저장
submission_file = 'C:/Users/82106/Desktop/데이터분석 프로젝트/대구 교통사고 피해 예측 AI 경진대회/open/open/submission_BayesianRidge.csv'
sample_submission_df.to_csv(submission_file, index=False)
# 제출 파일 경로 반환
submission_file

'C:/Users/82106/Desktop/데이터분석 프로젝트/대구 교통사고 피해 예측 AI 경진대회/open/open/submission_BayesianRidge.csv'

### 랜덤포레스트

In [18]:
from sklearn.ensemble import RandomForestRegressor

# 랜덤 포레스트 회귀 모델 생성
rf_model = RandomForestRegressor()

# 모델 훈련
rf_model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred_rf = rf_model.predict(X_test)

# 성능 평가
mse_rf = mean_squared_error(y_test, y_pred_rf)


In [19]:
mse_rf

0.07789182024741227